# Homework 1 — Finding Textually Similar Documents

### Objective

In this assignment, we implement a system to **find textually similar documents** using:
- **Shingling** — converting text into sets of substrings,
- **MinHashing** — efficiently estimating Jaccard similarity,
- **Locality-Sensitive Hashing (LSH)** — identifying potentially similar document pairs.

We will test the system on a **small corpus of English recipe texts**,  
each describing a different dish (e.g., pasta, risotto, paella, soup, etc.).  
By comparing them, we can observe how the algorithms detect similarity
between recipes that share ingredients, structure, or vocabulary.

---
### Pipeline Summary

1. **Load and preprocess** text data from files in the `data/` directory.
2. **Create k-shingles** for each document.
3. **Compute exact Jaccard similarities** between all pairs of documents.
4. **Generate MinHash signatures** and compare them to estimate similarity.
5. **Optionally apply LSH** to find similar documents more efficiently.
6. **Discuss results** and interpret which recipes are most similar.


In [ ]:
# --- Setup and imports ---

import os
import sys

# Add src/ folder to path for module imports
sys.path.append("../src")

# Import our custom functions
from shingling import create_shingles, hash_shingles
from compare_sets import jaccard_similarity
from minhashing import generate_hash_functions, compute_minhash_signature
from compare_signatures import signature_similarity
from lsh import lsh_candidate_pairs

# Optional: for visualizations later
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

print("Modules successfully imported.")


## 1. Loading and Inspecting the Dataset

We first load our **corpus of recipes** from the `data/` folder.  
Each file contains one recipe written in English — for example:
- *Pasta Carbonara*
- *Risotto alla Milanese*
- *Chicken Soup*
- *Spanish Paella*
- *Beef Stew*
- *Vegetable Curry*

By using texts from **different cuisines and types of dishes**,  
we can later verify whether the similarity detection methods
capture linguistic and semantic overlap (e.g., recipes for pasta may
be more similar to each other than to soups or desserts).


In [ ]:
# --- Load all text files from the data directory ---

data_path = "../data/"
documents = []
filenames = []

for fname in sorted(os.listdir(data_path)):
    if fname.endswith(".txt"):
        with open(os.path.join(data_path, fname), "r", encoding="utf-8") as f:
            documents.append(f.read())
            filenames.append(fname)

print(f"{len(documents)} documents loaded.\n")
print("Document names:", filenames)
print("\nSample text from the first recipe:\n")
print(documents[0][:400])

## 2. Shingling: Representing Documents as Sets

**Shingling** is the process of breaking each document into overlapping  
substrings (called *k-shingles*) of a fixed length `k`.

In [ ]:
# --- Create and hash k-shingles ---

k = 10  # shingle length (characters)
shingle_sets = []

for doc in documents:
    shingles = create_shingles(doc, k)
    hashed = hash_shingles(shingles)
    shingle_sets.append(hashed)

print(f"Created {len(shingle_sets)} shingle sets (one per recipe).")
print(f"Example shingles from first document: {list(create_shingles(documents[0], 10))[:10]}")


In [ ]:
n = len(shingle_sets)
jaccard_matrix = [[0]*n for _ in range(n)]

for i in range(n):
    for j in range(n):
        if i < j:
            sim = jaccard_similarity(shingle_sets[i], shingle_sets[j])
            jaccard_matrix[i][j] = jaccard_matrix[j][i] = sim

# Convert to DataFrame for readability
df_jaccard = pd.DataFrame(jaccard_matrix, index=filenames, columns=filenames)

print("Exact Jaccard similarity matrix:")
df_jaccard.round(3)


In [ ]:
# --- Visualize Jaccard similarities ---

plt.figure(figsize=(8,6))
sns.heatmap(df_jaccard, annot=True, fmt=".2f", cmap="Blues")
plt.title("Exact Jaccard Similarity Between Recipes")
plt.show()


## 4. MinHashing: Efficient Similarity Estimation

Computing Jaccard similarity directly can be slow for large datasets.  
**MinHashing** provides a way to approximate it efficiently.

The key idea:
- We define several random hash functions.
- Each hash function maps the shingles of a document to integers.
- The **minimum hash value** per function is recorded.
- The resulting vector of minimum values is the document’s **MinHash signature**.

The similarity between two signatures (fraction of equal positions)
approximates their Jaccard similarity.


In [ ]:
num_hashes = 100
max_shingle_id = 2**32 - 1  # modulus for hash functions

# Generate random hash functions
hash_functions = generate_hash_functions(num_hashes, max_shingle_id)

# Compute signatures for all documents
signatures = []
for s in shingle_sets:
    sig = compute_minhash_signature(s, hash_functions, max_shingle_id)
    signatures.append(sig)

print(f"Computed {len(signatures)} MinHash signatures (length {num_hashes} each).")


In [ ]:
# --- Estimate similarity using MinHash signatures ---

minhash_matrix = [[0]*n for _ in range(n)]

for i in range(n):
    for j in range(n):
        if i < j:
            sim = signature_similarity(signatures[i], signatures[j])
            minhash_matrix[i][j] = minhash_matrix[j][i] = sim

df_minhash = pd.DataFrame(minhash_matrix, index=filenames, columns=filenames)

print("Estimated similarities using MinHash:")
df_minhash.round(3)


In [ ]:
# --- Visualize MinHash-based similarity ---

plt.figure(figsize=(8,6))
sns.heatmap(df_minhash, annot=True, fmt=".2f", cmap="Greens")
plt.title("Estimated Similarity Using MinHash")
plt.show()


## 5. Locality-Sensitive Hashing (LSH)

To further reduce comparisons between documents,  
**Locality-Sensitive Hashing (LSH)** groups signatures into bands of rows.

Each band is hashed into a *bucket* — documents landing in the same bucket
are considered *candidate pairs*, likely to be similar.

This technique is widely used for large-scale similarity search
(e.g., near-duplicate detection on the web).


In [ ]:
bands = 20
rows = 5  # 20 * 5 = 100 (same as number of hash functions)

candidates = lsh_candidate_pairs(signatures, bands, rows)

print("Candidate pairs identified using LSH:")
for (i, j) in candidates:
    print(f"{filenames[i]}  <-->  {filenames[j]}")


## 6. Discussion and Interpretation

The results highlight how documents with overlapping wording or similar
content structure tend to show higher similarity values.

Once the recipe texts are finalized, this section should describe which
types of recipes appear most similar. For example:
- `[Recipe A]` and `[Recipe B]` might show high similarity due to shared
  ingredients or preparation steps.
- `[Recipe C]` might differ significantly because it uses a different cooking
  style or vocabulary.

When the actual dataset is available, you can replace these placeholders
with concrete observations based on the similarity matrices or LSH results.

---

### Insights to Include Later

- Which documents form small *clusters* of similar recipes?
- Are there recipes that are unexpectedly similar or dissimilar?
- Does the similarity correspond to the *type of dish*, *ingredients*, or *cooking method*?

---

### Scalability and Method Comparison

| Method | Description | Pros | Cons |
|:-------|:-------------|:------|:------|
| **Jaccard Similarity** | Exact overlap between sets | Intuitive and precise | Slow for large datasets |
| **MinHashing** | Approximation of Jaccard using random hashes | Fast, scalable | Slight approximation error |
| **LSH** | Groups similar signatures into candidate buckets | Very efficient for big data | May produce false positives |

---

### Conclusion

- **Shingling** transforms documents into comparable feature sets.  
- **Jaccard similarity** provides the ground truth of overlap.  
- **MinHashing** offers a scalable approximation with similar results.  
- **LSH** helps to quickly identify potential matches without
  exhaustively comparing every pair.

When the dataset is complete, these methods will reveal
how textual similarity reflects common patterns in the recipe corpus.